In [1]:
import requests
import json
import pandas as pd
import datetime
from datetime import datetime, timezone
from pathlib import Path
import hashlib
from sqlalchemy import create_engine
import numpy as np
from time import sleep
import logging
import pytz
import math
import os

In [2]:
from dotenv import load_dotenv
env_file = Path("../.env")
print(".env exists:", env_file.exists(), "→", env_file)
# Загружаем .env
loaded = load_dotenv(dotenv_path=env_file)

.env exists: True → ..\.env


In [3]:
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', 1000)  # Увеличить ширину отображения

In [5]:
# Текущее время
current_time = datetime.now(timezone.utc).isoformat()
current_time

'2025-12-08T07:50:16.901494+00:00'

In [6]:
url = os.getenv('URL')
headers = {
    'Content-Type': os.getenv('CONTENT_TYPE'),
    'Authorization': os.getenv('AUTHORIZATION'),
}

In [7]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

Накладные в статусе "в пути

In [8]:
payload = json.dumps({
  "com": "execOperation",
  "op": "static/getList()",
  "otype": "Invoice",
  "opargs": {
    "filters": [
          {
        "column": "lcState",
        "condition": "=",
        "value": [
         'onWay'
        ]
      },
      {
         "column": "recieverContractorId",
         "condition": "in",
         "value": [248824]
            }
    ],
    "size": 200,
    "getFullCards": 0
  }
}
)


response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)

invoice_data = response.text

#print(invoice_data)

data = json.loads(invoice_data)

# Извлечем таблицу из данных
attr_table = data["resData"]["attrTable"]

# Преобразуем в DataFrame
df = pd.DataFrame(attr_table[1:], columns=attr_table[0])

display(df)

200


,id,docNum,docDate,name,lcState,contractNum,ownerContractorId,doNotChangeProprietor,shipperContractorId,sourceWarehouseId,dateReceive,receiverContractorId,destinationWarehouseId,dateReturn,dateExport,carrierContractorId,vehicleNumber,docNote,docNoteReceiver,sysGuid,sysTimeFrom,sysChangedAt,ownerContractorName,sourceWarehouseName,receiverContractorName,destinationWarehouseName
0,4842974,908766,2025-12-05T09:00Z,908766 от 05.12.2025,onWay,,248824,,,1077174,2025-12-07T18:39:55.010Z,248824,0,,,,,,,,1765132843188,1765179826121,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
1,4843011,20251208-1,2025-12-08T07:41:44.321Z,20251208-1 от 08.12.2025,onWay,,248824,,,1077174,2025-12-08T07:41:46.195Z,248824,0,,,889,В390СО197,,,,1765179788176,1765179790114,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
2,4843012,20251208-2,2025-12-08T07:45:54.625Z,20251208-2 от 08.12.2025,onWay,,248824,,,1077174,2025-12-08T07:45:55.290Z,248824,0,,,1000,В299СО197,,,,1765179991982,1765179993611,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
3,4843013,20251208-3,2025-12-08T07:48:31.022Z,20251208-3 от 08.12.2025,onWay,,248824,,,1077174,,248824,1076945,,,1000,В391СО197,,,,1765180167954,1765180169985,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""","(RU-077/LU85248) 20_ТЦ ""Леруа Мерлен"" (Лефортово)"


In [9]:
#df.info()

Принятие накладных, минимально необходимые параметры.

In [ ]:
payload = json.dumps({
  "com": "execOperation",
  "op": "onWay/notifyDelivered()",
  "oid": "4817478", #  id накладной
  "otype": "Invoice",
  "opargs": {
    'theCard': 
    {'dateAction': current_time,
     'description': 'test_2',
     'destinationWarehouseId': 1077174 # без этого значения сервис вернет 500
     }
     }
}
)

headers = HEADERS

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)

Логика и цикл сервиса автоматического принятия накладных, последовательные вызовы для их принятия. Заложен timeout между вызовами в 0,5 секунд.

In [12]:
# Счетчики для статистики
success_count = 0
error_count = 0
skipped_count = 0

In [13]:
for index, row in df.iterrows():
    invoice_id = row['id']
    destination_warehouse_id = row['destinationWarehouseId']
    
    # Пропускаем записи с destinationWarehouseId = 0
    if destination_warehouse_id == 0:
        logger.warning(f"Пропущена запись {invoice_id}: destinationWarehouseId = 0")
        skipped_count += 1
        continue
    
    # Проверяем что значения не пустые
    if pd.isna(invoice_id) or pd.isna(destination_warehouse_id):
        logger.warning(f"Пропущена запись с индексом {index}: пустые значения")
        skipped_count += 1
        continue
    
    try:
        # Формируем payload
        payload = json.dumps({
            "com": "execOperation",
            "op": "onWay/notifyDelivered()",
            "oid": str(invoice_id),
            "otype": "Invoice", 
            "opargs": {
                'theCard': {
                    'dateAction': current_time,
                    'description': 'test_2',
                    'destinationWarehouseId': int(destination_warehouse_id)
                }
            }
        })
        
        # Запрос
        response = requests.request("POST", url, headers=headers, data=payload, timeout=30)
        
        # Логируем 
        logger.info(f"[{index + 1}/{len(df)}] ID: {invoice_id}, Склад: {destination_warehouse_id}, Status: {response.status_code}")
        
        if response.status_code == 200:
            success_count += 1
            try:
                result = response.json()
                logger.info(f"Принято: {result.get('resMsg', 'OK')}")
            except Exception as e:
                logger.error(f"Ошибка парсинга JSON: {e}")
        else:
            error_count += 1
            logger.error(f"HTTP ошибка: {response.text}")
            
    except requests.exceptions.RequestException as e:
        error_count += 1
        logger.error(f"Ошибка запроса для ID {invoice_id}: {e}")
    except Exception as e:
        error_count += 1
        logger.error(f"Неожиданная ошибка для ID {invoice_id}: {e}")
    
    # Пауза между запросами
    sleep(0.5)

2025-12-05 16:21:27,045 - INFO - [1/3] ID: 4817451, Склад: 4751, Status: 200
2025-12-05 16:21:27,045 - INFO - Принято: Ok
2025-12-05 16:21:27,710 - INFO - [2/3] ID: 4842897, Склад: 4751, Status: 200
2025-12-05 16:21:27,726 - INFO - Принято: Ok
2025-12-05 16:21:28,408 - INFO - [3/3] ID: 4842898, Склад: 1076971, Status: 200
2025-12-05 16:21:28,409 - INFO - Принято: Ok


In [14]:
# Итоговая статистика
logger.info(f"Обработка завершена. Успешно: {success_count}, Ошибок: {error_count}, Пропущено: {skipped_count}")

2025-12-05 16:21:41,981 - INFO - Обработка завершена. Успешно: 3, Ошибок: 0, Пропущено: 0
